In [14]:
import math
from datetime import date
import pickle
import os
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
from sys import stderr
from datetime import timedelta
import time
from ipywidgets import widgets
from IPython.display import display
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [27]:
class Auto:
    def __init__(self, marka, model, mesto, cena, link, godina, broj_oglasa=None):
        self._broj_oglasa = broj_oglasa
        self._marka = marka
        self._godina = godina
        self._model = model
        self._mesto = mesto
        self._cena = cena
        self._link = link
    def __str__(self):
        return f'Marka: {self._marka} model: {self._model} mesto: {self._mesto} cena: {self._cena}, godina: {self._godina}'


    def __eq__(self, other):
        if not isinstance(other, Auto):
            return False
        return self.broj_oglasa == other.broj_oglasa
    
    
    @property
    def godina(self):
        return self._godina
    
    @godina.setter
    def godina(self, value):
        self._godina = value
        
    @property
    def link(self):
        return self._link

    @link.setter
    def link(self, value):
        self._link = value
        
    @property
    def broj_oglasa(self):
        return self._broj_oglasa

    @broj_oglasa.setter
    def broj_oglasa(self, value):
        self._broj_oglasa = value

    @property
    def marka(self):
        return self._marka

    @marka.setter
    def marka(self, value):
        self._marka = value

    @property
    def mesto(self):
        return self._mesto

    @mesto.setter
    def mesto(self, value):
        self._mesto = value

    @property
    def model(self):
        return self._model

    @model.setter
    def model(self, value):
        self._model = value

    @property
    def cena(self):
        return self._cena

    @cena.setter
    def cena(self, value):
        self._cena = value


In [3]:
def get_chrome_web_driver():
    options = ChromeOptions()
    options.add_argument('headless') # to prevent browser from opening when extracting content of a web page
    return webdriver.Chrome(options=options)

In [4]:
def citanje_stranice(driver, url):
    lista=list()
    driver.get(url)
    page_content = driver.page_source
    page_soup = BeautifulSoup(page_content, features='html.parser')
    if not page_soup:
        return None
    return page_soup

In [5]:
def izvlacenje_automobila_sa_stranice(supa):
    lista=list()
    supa=supa.findAll('div', class_='js-hide-on-filter')[1]
    lista_polja=supa.findAll('article')
    lista_polja=supa.select('article.classified')
    for p in lista_polja:
        podaci=p.find('div', class_='textContent')#title i href
        a=podaci.find('a')
        naslov=a.get('title')
        naslov=naslov.split(" ")
        marka=naslov[0]##
        naslov.pop(0)
        model=" ".join(naslov)##
        link=a.get('href')##pisalo podaci
        oglas=link.split('/')[2]##
        
        link=f'https://www.polovniautomobili.com{link}'
       # /auto-oglasi/25271868/bmw-i3-94-ah?attp=p1_pv0_pc1_pl1_plv0
        c=podaci.find("div", class_="price")
        cena=c.text.strip().split(" ")[0] ##
        godina=podaci.find('div', class_='setInfo').find('div', class_='top').get('title').split(" ")[0]##
        mesto=p.find('div', class_='city').text.strip()##
        auto=Auto(marka,model,mesto,cena,link,godina)
        auto.broj_oglasa=oglas
        lista.append(auto)
    return lista
    

In [38]:
def citanje_fajla(fajl):
    #fajl = "lista.bin"
    if os.path.exists(fajl):
        try:
            with open(fajl, "rb") as binarna:
                lista = pickle.load(binarna)
        except EOFError:
            lista = None
            print("A")
    else:
        lista = None
        print("B")
    return lista

In [7]:
def pisanje_u_fajl(fajl, lista):#"lista.bin" je fajl
    with open(fajl, "wb") as binarna:
        pickle.dump(lista, binarna)

In [16]:
url="https://www.polovniautomobili.com/auto-oglasi/pretraga?brand=&brand2=&price_from=&price_to=20000&year_from=&year_to=&fuel%5B%5D=2312&flywheel=&atest=&door_num=&submit_1=&without_price=1&date_limit=&showOldNew=all&modeltxt=&engine_volume_from=&engine_volume_to=&power_from=&power_to=&mileage_from=&mileage_to=&emission_class=&seat_num=&wheel_side=&registration=&country=&country_origin=&city=&registration_price=&page=10&sort=basic"
try:
    driver = get_chrome_web_driver()
    if not driver:
        raise RuntimeError("An error occurred while setting up web driver!")
    supa=citanje_stranice(driver, url)
    supa=supa.findAll('div', class_='js-hide-on-filter')[1]
    if supa:
        supa=supa.find('small')
        broj_oglasa=int(supa.text.split(" ")[-1])
        print("broj oglasa: "+str(broj_oglasa))
        broj_stranica=math.ceil(broj_oglasa/25)
        print(f'broj stranica: {broj_stranica}')
except Exception as e:
    raise e
    #print(e)
        #<small>Prikazano od 1 do 25 oglasa od ukupno 230</small>

broj oglasa: 230
broj stranica: 10


In [34]:
try:
    listaAutomobila=[]
    for i in range(1, broj_stranica+1):
        link=f'https://www.polovniautomobili.com/auto-oglasi/pretraga?page={i}&sort=basic&price_to=20000&fuel%5B0%5D=2312&city_distance=0&showOldNew=all&without_price=1'
        #print(link)
        supa=citanje_stranice(driver, link)
        if supa:
            listaAutomobila.extend(izvlacenje_automobila_sa_stranice(supa))
        else:
            raise Exception("GRESKA1")
    print("Lista automobila:")
    broj=1
    for a in listaAutomobila:
        print(str(broj)+". "+str(a))
        print(a.link)
        print(a.broj_oglasa)
        broj=broj+1
except RuntimeError as err:
    stderr.write(f"Terminating the program due to the following runtime error:\n{err}")
except Exception as err:
    print("Greska")
    raise err
    stderr.write(f"Terminating the program due to the following runtime error:\n{err}")

Lista automobila:
1. Marka: Smart model: ForTwo KREDlTl BEZ UCESCA mesto: Beograd cena: 12.499, godina: 2020.
https://www.polovniautomobili.com/auto-oglasi/25179331/smart-fortwo-kredltl-bez-ucesca?attp=p1_pv0_pc1_pl1_plv0
25179331
2. Marka: BMW model: i3 120Ah ADVANCED 170Hp mesto: Novi Beograd cena: 18.990, godina: 2019.
https://www.polovniautomobili.com/auto-oglasi/25155739/bmw-i3-120ah-advanced-170hp?attp=p1_pv0_pc1_pl1_plv0
25155739
3. Marka: Peugeot model: iOn iOn Electric mesto: Subotica cena: 8.000, godina: 2015.
https://www.polovniautomobili.com/auto-oglasi/25605287/peugeot-ion-ion-electric?attp=p3_pv0_pc1_pl1_plv0
25605287
4. Marka: Renault model: Zoe BOSE/47.500k/FUL mesto: Valjevo cena: 12.350, godina: 2016.
https://www.polovniautomobili.com/auto-oglasi/25259985/renault-zoe-bose47500kful?attp=p1_pv0_pc1_pl1_plv0
25259985
5. Marka: Smart model: ForTwo EQ BRABUS/CABRIO/FUL mesto: Valjevo cena: 14.990, godina: 2017.
https://www.polovniautomobili.com/auto-oglasi/25259964/smart-f

In [42]:
stara_lista_automobila=citanje_fajla("elektricni.bin")
print(f'Prosli put je bilo {len(stara_lista_automobila)} automobila')

Prosli put je bilo 229 automobila


In [41]:
pisanje_u_fajl("elektricni.bin", listaAutomobila)

In [43]:

lista_novih_automobila=[]
for a in listaAutomobila:
    if not a in stara_lista_automobila:
        lista_novih_automobila.append(a)
if(len(lista_novih_automobila)==0):
    print("Nema novih automobila")
else:
    print(f'Ima {len(lista_novih_automobila)} novih automobila')
    print("Novi automobili")
    for a in lista_novih_automobila:
        print(a)

Nema novih automobila
